# Interactive Audit Testing

This notebook allows for real-time testing of the IFRS9 Automated Auditor without using web ports.

In [1]:
# Cell 1: Load environment and initialize RcmAuditor
import os
from dotenv import load_dotenv
from rcm_engine import RcmAuditor
from config import CONFIG

load_dotenv()

# Initialize Auditor
auditor = RcmAuditor()
print("RcmAuditor initialized.")

# Ensure documents are indexed
auditor.initialize_rag()
print("RAG Index ready.")

RcmAuditor initialized.
Loading documents from documents/...
Loading Política de Previsionamiento_modificada sep 2025.pdf...
Loading Respuesta Memorando final Inspección 2025_Perdida Esperada NIIF.pdf...
Creating vector store with 58 chunks (Size: 1500, Overlap: 300)...
Index built successfully.
RAG Index ready.


In [2]:
# Cell 1.5: Dynamic Language Configuration
current_lang = auditor.rag_engine.doc_language
print(f"Current Language: {current_lang}")

# Uncomment to change language dynamically and re-run queries
# new_lang = "English"
# auditor.rag_engine.doc_language = new_lang
# print(f"Language updated to: {auditor.rag_engine.doc_language}")

Current Language: Spanish


In [3]:
# Cell 2: Define a specific test question manually
test_question = "Is lifetime PD estimation well explained and consistent with the expected life of exposures and staging approach?"#"Como se calcula la PD a 12 meses?"
print(f"Test Question: {test_question}")

Test Question: Is lifetime PD estimation well explained and consistent with the expected life of exposures and staging approach?


In [4]:
# Cell 3: Run the retrieval step only and print retrieved PDF text chunks
print("Retrieving top 5 chunks...")

# Verify HyDE generation
print(f"Generating HyDE query for: '{test_question}'")
hyde_query = auditor.rag_engine.generate_search_query(test_question)
print(f"HyDE Output ({auditor.rag_engine.doc_language}):\n{hyde_query}\n")

retrieved_docs = auditor.rag_engine.retrieve(test_question, k=5)

for i, doc in enumerate(retrieved_docs):
    print(f"\n--- Chunk {i+1} (Page {doc.metadata.get('page', 'N/A')}) ---")
    print(doc.page_content[:500] + "...") # Print first 500 chars

Retrieving top 5 chunks...
Generating HyDE query for: 'Is lifetime PD estimation well explained and consistent with the expected life of exposures and staging approach?'
HyDE Output (Spanish):
La estimación de la Probabilidad de Incumplimiento (PD) a lo largo de la vida útil está adecuadamente explicada y es consistente con la vida esperada de las exposiciones y el enfoque de etapas. El modelo utilizado para calcular la PD incorpora factores macroeconómicos y específicos del deudor que afectan la probabilidad de incumplimiento a lo largo del tiempo. Además, se han implementado metodologías robustas para segmentar las exposiciones en diferentes etapas, asegurando que la PD refleje adecuadamente el riesgo crediticio en cada fase del ciclo de vida del activo. La coherencia entre la estimación de la PD y el enfoque de etapas se verifica mediante pruebas de validación interna y análisis de sensibilidad, garantizando que las proyecciones sean razonables y alineadas con las expectativas del m

In [ ]:
# Cell 4: Run the full process_row logic and print AI Answer AND Critique Score

# Mock a row data structure
mock_row = {
    'Control Reference': 'Test-Ref-001',
    'Test Procedure': test_question
}

print("Running full process_row...")
result = auditor.process_row(mock_row)

print("\n=== AI Answer ===")
print(result['AI_Answer'])

print("\n=== Validation ===")
print(f"Score: {result['Validation_Score']}")
print(f"Reasoning: {result['Validation_Reasoning']}")

Running full process_row...
DEBUG: Generating HyDE query in Spanish...
Original Query: Control Ref: Test-Ref-001. Procedure: La pd se cal...
HyDE Search Query: La pérdida esperada (pd) se calcula principalmente...

=== AI Answer ===
The Probability of Default (PD) is calculated based on the number of cases (individual exposures) rather than the amounts. However, there are exceptions for the segments "Descuento de cheques" and "cuentas corrientes," where it is determined by amounts. This is documented on Page 6.

=== Validation ===
Score: 10


KeyError: 'Hallucination_Flag'